<h1>Analysis of Global COVID-19 Pandemic Data</h1>






## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [1]:
# This lab requires 'httr' and 'rvest'packages, which are already pre-loaded into this lab environment.
# However, if you are working on your local RStudio, please uncomment the below codes and install the packages.

#install.packages("httr")
#install.packages("rvest")

Warning message in install.packages("httr"):
"'lib = "C:/Program Files/R/R-4.3.2/library"' is not writable"


ERROR: Error in install.packages("httr"): unable to install packages


In [1]:
library(httr)
library(rvest)

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this 

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [ ]:
get_wiki_covid19_page <- function() {
  # Wiki page base
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
  
  # URL parameters
  url_parameters <- list(title = "Template:COVID-19_testing_by_country")
  
  # Use the `GET` function in httr library with a `url` argument and a `query` argument to get a HTTP response
  response <- httr::GET(url = wiki_base_url, query = url_parameters)
  
  # Use the `return` function to return the response
  return(response)
}


: 

Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [7]:
# Call the get_wiki_covid19_page function and print the response
# Call the function
response <- get_wiki_covid19_page()

# Print the response
print(response)


Response [https://en.wikipedia.org/w/index.php?title=Template%3ACOVID-19_testing_by_country]
  Date: 2023-12-30 02:41
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 448 kB
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-fea...
<head>
<meta charset="UTF-8">
<title>Template:COVID-19 testing by country - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-heade...
"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","...
"CS1 Russian-language sources (ru)","CS1 Bosnian-language sources (bs)","CS1 ...
"CS1 Malagasy-language sources (mg)","CS1 Malay-language sources (ms)","CS1 R...
"wgIsProbablyEditable":false,"wgRelevantPageIsProbablyEditable":false,"wgRest...
...


##  Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [8]:
# Load rvest package
library(rvest)

# Parse the HTTP response with read_html
root_html_node <- read_html(response)

# Print the root HTML node
print(root_html_node)


{html_document}
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-toc-available" lang="en" dir="ltr">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="skin-vector skin-vector-search-vue mediawiki ltr sitedir-ltr ...


Get the tables in the HTML root node using `html_nodes` function.


In [9]:
# Extract tables from the root HTML node
tables <- html_nodes(root_html_node, "table")

# Print the tables
print(tables)


{xml_nodeset (4)}
[1] <table class="box-Update plainlinks ombox ombox-content ambox-Update" rol ...
[2] <table class="wikitable plainrowheaders sortable collapsible autocollapse ...
[3] <table class="plainlinks ombox mbox-small ombox-notice" role="presentatio ...
[4] <table class="wikitable mw-templatedata-doc-params">\n<caption><p class=" ...


Read the specific table from the multiple tables in the `table_node` using the `html_table` function and convert it into dataframe using `as.data.frame`

_Hint:- Please read the `table_node` with index 2(ex:- table_node[2])._


In [12]:
# Read the table at index 2
table_2 <- html_table(tables[2])

# Convert the table into a dataframe
df <- as.data.frame(table_2)

# Print the dataframe
print(df)


##  Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [11]:
# Print the summary of the dataframe
summary(df)


 Country.or.region    Date.a.             Tested            Units.b.        
 Length:173         Length:173         Length:173         Length:173        
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
 Confirmed.cases.   Confirmed..tested.. Tested..population..
 Length:173         Length:173          Length:173          
 Class :character   Class :character    Class :character    
 Mode  :character   Mode  :character    Mode  :character    
 Confirmed..population..     Ref.          
 Length:173              Length:173        
 Class :character        Class :character  
 Mode  :character        Mode  :character  

As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`. 

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [13]:
preprocess_covid_data_frame <- function(data_frame) {
    
    shape <- dim(data_frame)

    # Remove the World row
    data_frame<-data_frame[!(data_frame$`Country.or.region`=="World"),]
    # Remove the last row
    data_frame <- data_frame[1:172, ]
    
    # We dont need the Units and Ref columns, so can be removed
    data_frame["Ref."] <- NULL
    data_frame["Units.b."] <- NULL
    
    # Renaming the columns
    names(data_frame) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")
    
    # Convert column data types
    data_frame$country <- as.factor(data_frame$country)
    data_frame$date <- as.factor(data_frame$date)
    data_frame$tested <- as.numeric(gsub(",","",data_frame$tested))
    data_frame$confirmed <- as.numeric(gsub(",","",data_frame$confirmed))
    data_frame$'confirmed.tested.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.tested.ratio`))
    data_frame$'tested.population.ratio' <- as.numeric(gsub(",","",data_frame$`tested.population.ratio`))
    data_frame$'confirmed.population.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.population.ratio`))
    
    return(data_frame)
}


Call the `preprocess_covid_data_frame` function


In [14]:
# Call the preprocess_covid_data_frame function
preprocessed_data_frame <- preprocess_covid_data_frame(df)

# Print the preprocessed dataframe
print(preprocessed_data_frame)


                   country        date    tested confirmed
1              Afghanistan 17 Dec 2020    154767     49621
2                  Albania 18 Feb 2021    428654     96838
3                  Algeria  2 Nov 2020    230553     58574
4                  Andorra 23 Feb 2022    300307     37958
5                   Angola  2 Feb 2021    399228     20981
6      Antigua and Barbuda  6 Mar 2021     15268       832
7                Argentina 16 Apr 2022  35716069   9060495
8                  Armenia 29 May 2022   3099602    422963
9                Australia  9 Sep 2022  78548492  10112229
10                 Austria  1 Feb 2023 205817752   5789991
11              Azerbaijan 11 May 2022   6838458    792638
12                 Bahamas 28 Nov 2022    259366     37483
13                 Bahrain  3 Dec 2022  10578766    696614
14              Bangladesh 24 Jul 2021   7417714   1151644
15                Barbados 14 Oct 2022    770100    103014
16                 Belarus  9 May 2022  13217569    9828

In [15]:
# Print the summary of the preprocessed dataframe
summary(preprocessed_data_frame)


                country             date         tested         
 Afghanistan        :  1   2 Feb 2023 :  6   Min.   :     3880  
 Albania            :  1   1 Feb 2023 :  4   1st Qu.:   512037  
 Algeria            :  1   31 Jan 2023:  4   Median :  3029859  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 31377219  
 Angola             :  1   23 Jul 2021:  3   3rd Qu.: 12386725  
 Antigua and Barbuda:  1   29 Jan 2023:  3   Max.   :929349291  
 (Other)            :166   (Other)    :149                      
   confirmed        confirmed.tested.ratio tested.population.ratio
 Min.   :       0   Min.   : 0.00          Min.   :   0.006       
 1st Qu.:   37839   1st Qu.: 5.00          1st Qu.:   9.475       
 Median :  281196   Median :10.05          Median :  46.950       
 Mean   : 2508340   Mean   :11.25          Mean   : 175.504       
 3rd Qu.: 1278105   3rd Qu.:15.25          3rd Qu.: 156.500       
 Max.   :90749469   Max.   :46.80          Max.   :3223.000       
           

After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file. 


In [19]:
# Write the preprocessed dataframe into a CSV file
write.csv(preprocessed_data_frame, file = "preprocessed_covid_data.csv")


Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [24]:
# Get working directory
wd <- getwd()
# Get exported 
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

[1] "c:/Users/sheha/Documents/Code/Project/R Covid/covid.csv"


[1] FALSE

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [ ]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

##   Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [27]:
# Read the CSV file
covid_data_frame_csv <- read.csv("preprocessed_covid_data.csv")

# Get the 5th to 10th rows, with "country" and "confirmed" columns
selected_data <- covid_data_frame_csv[5:10, c("country", "confirmed")]

# Print the selected data
print(selected_data)


               country confirmed
5               Angola     20981
6  Antigua and Barbuda       832
7            Argentina   9060495
8              Armenia    422963
9            Australia  10112229
10             Austria   5789991


##  Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [ ]:
# Get the total confirmed cases worldwide

# Get the total tested cases worldwide

# Get the positive ratio (confirmed / tested)


##  Get a country list which reported their testing data 

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [28]:
# Get the total confirmed cases worldwide
total_confirmed <- sum(covid_data_frame_csv$confirmed, na.rm = TRUE)

# Get the total tested cases worldwide
total_tested <- sum(covid_data_frame_csv$tested, na.rm = TRUE)

# Get the positive ratio (confirmed / tested)
positive_ratio <- total_confirmed / total_tested

# Print the results
print(paste("Total confirmed cases worldwide: ", total_confirmed))
print(paste("Total tested cases worldwide: ", total_tested))
print(paste("Positive ratio (confirmed / tested): ", positive_ratio))


[1] "Total confirmed cases worldwide:  431434555"
[1] "Total tested cases worldwide:  5396881644"
[1] "Positive ratio (confirmed / tested):  0.0799414520197323"


##  Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [29]:
# Use a regular expression to find matches
matches <- grep("United.+", covid_data_frame_csv$country, value = TRUE)

# Print the matched country names
print(matches)


[1] "United Arab Emirates" "United Kingdom"       "United States"       


##   Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [33]:
# Select countries
selected_countries <- c("United States", "Japan")  # replace with your selected countries

# Select columns
selected_columns <- c("country", "confirmed", "confirmed.population.ratio")  # replace with your selected columns

# Initialize an empty dataframe to store the selected data
selected_data <- data.frame()

# Loop through each selected country
for (country in selected_countries) {
  # Select a subset of the dataframe based on the current country and columns
  subset_df <- subset(covid_data_frame_csv, country == country, select = selected_columns)
  
  # Append the subset dataframe to the selected data dataframe
  selected_data <- rbind(selected_data, subset_df)
}

# Print the selected data
print(selected_data)


                   country confirmed confirmed.population.ratio
1              Afghanistan     49621                    0.13000
2                  Albania     96838                    3.40000
3                  Algeria     58574                    0.13000
4                  Andorra     37958                   49.00000
5                   Angola     20981                    0.06700
6      Antigua and Barbuda       832                    0.86000
7                Argentina   9060495                   20.00000
8                  Armenia    422963                   14.30000
9                Australia  10112229                   40.30000
10                 Austria   5789991                   65.00000
11              Azerbaijan    792638                    8.00000
12                 Bahamas     37483                    9.70000
13                 Bahrain    696614                   44.40000
14              Bangladesh   1151644                    0.70000
15                Barbados    103014    

##  Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [36]:
# Initialize variables to store the country name and confirmed population ratio
max_ratio_country <- ""
max_ratio <- 0

# Loop through each selected country
for (country in selected_countries) {
  # Get the confirmed population ratio for the current country
  ratio <- subset(covid_data_frame_csv, country == country)$confirmed.population.ratio
  
  # Loop through each element in ratio
  for (i in 1:length(ratio)) {
    # If the current element in ratio is greater than max_ratio
    if (ratio[i] > max_ratio) {
      # Update max_ratio and max_ratio_country
      max_ratio <- ratio[i]
      max_ratio_country <- country
    }
  }
}

# Print the country with the max confirmed population ratio
print(paste("The country with the max confirmed population ratio is: ", max_ratio_country))


[1] "The country with the max confirmed population ratio is:  United States"


##  Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [37]:
# Set the threshold
threshold <- 0.05  # replace with your threshold

# Get a subset of countries with confirmed.population.ratio less than the threshold
subset_df <- subset(covid_data_frame_csv, confirmed.population.ratio < threshold)

# Print the subset dataframe
print(subset_df)


      X          country        date    tested confirmed confirmed.tested.ratio
28   28          Burundi  5 Jan 2021     90019       884                  0.980
32   32             Chad  2 Mar 2021     99027      4020                  4.100
34   34         China[c] 31 Jul 2020 160000000     87655                  0.055
45   45         DR Congo 28 Feb 2021    124838     25961                 20.800
89   89             Laos  1 Mar 2021    114030        45                  0.039
104 104        Mauritius 22 Nov 2020    289552       494                  0.170
117 117            Niger 22 Feb 2021     79321      4740                  6.000
119 119      North Korea 25 Nov 2020     16914         0                  0.000
127 127 Papua New Guinea 17 Feb 2021     47490       961                  2.000
156 156         Tanzania 18 Nov 2020      3880       509                 13.100
157 157         Thailand  4 Mar 2021   1579597     26162                  1.700
    tested.population.ratio confirmed.po